In [ ]:
%run src/init_notebooks.py
hide_toggle()

# A GROMACS - HADDOCK workflow - Title

```
Author                : XXX
Goal                  : Provide a template for how to write a jupyter notebook for training purposes
Time                  : 10 minutes reading time, no computation wait time
Prerequisites         : Know how to run an md simulation
Software requirements : GROMACS (version 2020) , vmd
Tested for            : 
```
 

 # Use Case Introduction

 - Introduction of the use case 
   aim:
 
 - Specific problem 

 - Introduction of use case as tutorial
 what they will learn from a tutorial
  
 - References
     - reading material

# Description of the Workshop Flow

### To Do
- Description of the workshop 
  - 
- Add a picture of the workflow, gromacs, bioexcel 

![ImageNotFoundAlternativeTxt](images/Bioexcel_symbol.svg "Text you will see when hovering over the logo with the mouse")

![ImageNotFoundAlternativeTxt](images/gmx_logo_blue.svg "Logo Title")

![ImageNotFoundAlternativeTxt](images/non-existant.svg "Logo Title Text 1")
 



# System setting

The starting point is a antibody structure file. For this tutorial, we will utilize  XXXX (PDB code XXXX). This file is available from the RCSB website, https://www.rcsb.org/. For this tutorial, the PDB file for the crystal structure is depositied in this directory as "xxx.pdb" .

In [ ]:
# Change to the data directory
# Note that executing this command twice will result in an error you can ignore
%cd data

Below you can visualize the antibody structure  

In [ ]:
import nglview as ng
view = ng.show_file("input/xxx.pdb", default=False)
view
# click and drag to rotate, zoom with your mouseweel 
# for more infor on this viewer have a look at https://github.com/nglviewer/nglview

In alternative you can visualize the structure using a viewing program such as VMD.
Note: close the VMD window after you are done looking at the protein to continue with this notebook

In [ ]:
!vmd input/xxxx.pdb

Once you've had a look at the molecule, you are going to check that only the protein is present in the pdb file. Otherwise strip out all the other molecules in the crystal . To delete the other molecules , either use a plain text editor like vi, emacs (Linux/Mac), or Notepad (Windows). Do not use word processing software! 

Always check your .pdb file for entries listed under the comment MISSING, as these entries indicate either atoms or whole residues that are not present in the crystal structure. Terminal regions may be absent, and may not present a problem for dynamics.

## Generating a topology

Now the PDB file should contain only protein atoms, and is ready to be input into GROMACS. 
The first GROMACS tool, we use, is pdb2gmx. The purpose of pdb2gmx is to generate three files:

* The topology for the molecule.
* A position restraint file.
* A post-processed structure file. 

The topology (topol.top by default) contains all the information necessary to define the molecule within a simulation. This information includes nonbonded parameters (atom types and charges) as well as bonded parameters (bonds, angles, dihedrals and atom connectivity).

## Force Field

Here, we made an important decision for the course of the simualtion in choosing the force field. Here we use CHARMM36m all-atom force field (see (here) for update in the CHARMM36 force field implementation for GROMACS http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs. The force field files contain the information that will be written to the topology. 

In [ ]:
!tar xvf input/charmm36.tar

In [ ]:
!gmx pdb2gmx -f input/xxx.pdb -p antibody.top -o antibody.gro -i posre -ff charmm36-jul2017 -water tip3p -ignh

## defining simulation box

In [ ]:
!gmx editconf -f antibody.gro -d 0.7 -bt dodecahedron -o box.gro

## filling the box with water molecules

In [ ]:
!gmx solvate -cp box.gro -p antibody.top -o water.gro

## adding ions

In [ ]:
!touch ion.mdp
!gmx grompp -f ion.mdp -c water.gro -p antibody.top -o
!echo 13 | gmx genion -s topol.tpr -p antibody.top -neutral -conc 0.15 -o startMM.gro

# System equilibration

In [ ]:
!mkdir MM
!gmx grompp -f steep_charmm36m.mdp -c startMM.gro -p antibody.top -o MM/topol.tpr! 
!gmx mdrun -v -s MM/topol.tpr -deffnm antibody_MM
!mkdir POS
!gmx grompp -f md_eq_posre_charmm36m.mdp -c antibodyMM.gro -p antibody.top -o POS/topol.tpr 
!gmx mdrun -v -s POS/topol.tpr -deffnm antibody_P


cd ..
gmx grompp -f ../../../mdp_files/md_eq_posre_charmm36m.mdp -c MM/confout.gro  -r MM/confout.gro -p $name.top -o POS/topol.tpr
#gmx grompp -f ../../../mdp_files/md_charmm36m.mdp -c POS/confout.gro -p $name.top -o MD/topol.tpr

#cp topol.tpr in topol100.tpr
#gmx convert-tpr -s topol100.tpr -extend 400000 -o topol.tpr

## defining simulation box

# Design principles for the tutorials

# Writing the notebook

- Style the notebook with style sheet provided in src
    - Rationale: Easy way to set context for notebooks, create a "GROMACS"-tutorial feel
    - Con: requires users to execute the first cell
    - Pro: can serve as an introduction on how to execute cells

- Write headers (# title, ## header2, etc.) in seperate cells 
    - Rationale: this enables folding of sections 

- Include images as

![ImageNotFoundAlternativeTxt](images/Bioexcel_symbol.svg "Text you will see when hovering over the logo with the mouse")

![ImageNotFoundAlternativeTxt](images/non-existant.svg "Logo Title Text 1")


- Toggle solutions with the hide_toggle() function, hide the next cell with for_next=True

In [ ]:
#hide this cell
hide_toggle()

In [ ]:
hide_toggle(for_next=True)

In [ ]:
#some solution that should not yet be visible

more text

# Common pitfalls

- Cells are stuck in evaluation (marked with `[*]`)
    - Reason: Cells are evaluated serially. Jupyter cannot run bash commands in the background. Especially when opening another programm, like vmd, the notebook requires that the window is closed before going further
    - Solution : Make workshop participants aware of this 
    - Solution : Show users the kernel -> interrupt solution
    - Solution : start evaluations in subprocess
        - Con : spawning subprocesses from within notebooks depends on the python verion. 
        - Con : This requires some python boilerplate code that makes it harder to have a one-to-one correspondence of the command line command and what users will read in the notebook
- Users don't notice that they can execute / run input cells with the run botton, but rather copy-paste or type into the command line
- Users feel like they are taken through the notebooks via "auto-pilot"
    - Solution: Include the `hide_toggles()` function
    - Solution: Use quizzes
    - Con: This requires executing the first line of code in the notebook to work well
- The notebook is unaware of the changes to shell environment and variables changes when executed with '! command!
    - Reason: Bash commands are executed in subshells 
- '!cd directory' does not work as expected
    - Reason: Bash commands are executed in subshells 
    - Solution: use `%cd ` or `cd ` to permanently change the current directory
    - Con: this is notebook 'magic' that might confuse useres
- Loading environment modules does not work
    - Reason: Bash commands are executed in subshells
    - Solution: Use python module commands
       - Con : Depends on the python version and importing external modules
- Markdown cell line breaks look differnetly when markdown cells are executed
- Showing contents of long files requires lots of scrolling
  - Solution: use `%less filename`
- Longer bash script is hard to read in a cell
  - Solution: use `%bash` on top of the cell, then continue with usual bash